In [1]:
import pandas as pd

## 0 Read Data

In [2]:
# read user

In [3]:
path = 'E:/DSBA ESSEC - CentraleSupelec/Foundation of Machine Learning/美团商家推荐/原数据-20210301-20210328/users.txt'
users = pd.read_csv(path, sep = "\t")

In [4]:
users.head()
# users
# users.info()
# users['avg_pay_amt'].unique()

,user_id,avg_pay_amt,avg_pay_amt_weekdays,avg_pay_amt_weekends
0,0,"[36,49)","[29,36)","[36,49)"
1,1,"[49,65)","[49,65)",>=65
2,2,<29,<29,<29
3,3,"[49,65)",>=65,"[49,65)"
4,4,"[36,49)",未知,"[49,65)"


In [5]:
users['avg_pay_amt'].apply(lambda x: str(x).split(','))

0         [[36, 49)]
1         [[49, 65)]
2              [<29]
3         [[49, 65)]
4         [[36, 49)]
             ...    
199995    [[36, 49)]
199996        [>=65]
199997    [[36, 49)]
199998         [<29]
199999         [<29]
Name: avg_pay_amt, Length: 200000, dtype: object

In [6]:
# read pois

In [7]:
path = 'E:/DSBA ESSEC - CentraleSupelec/Foundation of Machine Learning/美团商家推荐/原数据-20210301-20210328/pois.txt'
pois = pd.read_csv(path, sep = "\t")

In [8]:
# pois.head()
# orders_train
pois.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29072 entries, 0 to 29071
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   wm_poi_id                   29071 non-null  float64
 1   wm_poi_name                 27656 non-null  object 
 2   primary_second_tag_name     27656 non-null  object 
 3   primary_third_tag_name      27656 non-null  object 
 4   primary_first_tag_name      27656 non-null  object 
 5   poi_brand_id                22037 non-null  float64
 6   aor_id                      4996 non-null   float64
 7   poi_score                   27076 non-null  float64
 8   delivery_comment_avg_score  27076 non-null  float64
 9   food_comment_avg_score      27076 non-null  float64
dtypes: float64(6), object(4)
memory usage: 2.2+ MB


In [9]:
# read orders

In [10]:
path = 'E:/DSBA ESSEC - CentraleSupelec/Foundation of Machine Learning/美团商家推荐/原数据-20210301-20210328/orders_train.txt'
orders_train = pd.read_csv(path, sep = "\t")

In [11]:
orders_train.head()
# orders_train

,user_id,wm_order_id,wm_poi_id,aor_id,order_price_interval,order_timestamp,ord_period_name,order_scene_name,aoi_id,takedlvr_aoi_type_name,dt
0,120037,0,1806,0,<29,1614557985,0,未知,0.0,0,20210301
1,135109,1,970,6,>=65,1614561844,1,0,NaN,未知,20210301
2,146736,2,1945,6,"[29,36)",1614588773,2,未知,1.0,0,20210301
3,83644,3,708,6,>=65,1614599916,3,1,2.0,0,20210301
4,117903,4,2366,6,"[29,36)",1614595418,3,1,3.0,1,20210301


## 1 Cleaning Data

In [12]:
# pois

In [13]:
pois = pois[0:29070]

## Map POIs to Integers (Done)

## 2 Exploring

### Most Bought POI

In [14]:
order_sort_by_poi = orders_train.groupby('wm_poi_id').count().sort_values('user_id',ascending = False)

In [15]:
order_sort_by_poi

,user_id,wm_order_id,aor_id,order_price_interval,order_timestamp,ord_period_name,order_scene_name,aoi_id,takedlvr_aoi_type_name,dt
wm_poi_id,,,,,,,,,,
1896,4535,4535,4535,4535,4535,4535,4535,4330,4535,4535
244,4139,4139,4139,4139,4139,4139,4139,3531,4139,4139
3230,4102,4102,4102,4102,4102,4102,4102,3766,4102,4102
3193,3943,3943,3943,3943,3943,3943,3943,3647,3943,3943
2933,3868,3868,3868,3868,3868,3868,3868,3737,3868,3868
...,...,...,...,...,...,...,...,...,...,...
1153,1,1,1,1,1,1,1,1,1,1
1154,1,1,1,1,1,1,1,1,1,1
1995,1,1,1,1,1,1,1,1,1,1


## Try Collaborative Filtering for POI

In [33]:
# user without info
users_n_info = users['user_id'].to_frame()
users_n_info = users_n_info [:1000]

In [34]:
# users_n_info

In [35]:
# order record
user_poi_buy = orders_train[['user_id','wm_poi_id']]
user_poi_buy = user_poi_buy [:3000]

In [36]:
user_poi_buy['if_buy'] = 1

In [37]:
# user_poi_buy

In [38]:
# user poi relationship 0 

In [39]:
user_poi_0 = pd.merge(users_n_info.assign(key=1), user_poi_buy['wm_poi_id'].to_frame().assign(key=1),on='key').drop('key',1)

C:\Users\AXE-YU~1\AppData\Local\Temp/ipykernel_16100/3713264701.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  user_poi_0 = pd.merge(users_n_info.assign(key=1), user_poi_buy['wm_poi_id'].to_frame().assign(key=1),on='key').drop('key',1)


In [40]:
user_poi_0['if_buy'] = 0

In [41]:
# user_poi_0

In [42]:
user_poi_if_buy = pd.merge(user_poi_0, user_poi_buy, on = ['user_id','wm_poi_id'], how = 'left')

In [43]:
user_poi_if_buy['if_buy'] = user_poi_if_buy['if_buy_x'] + user_poi_if_buy['if_buy_y']
user_poi_if_buy = user_poi_if_buy[['user_id','wm_poi_id','if_buy']]
user_poi_if_buy = user_poi_if_buy.fillna(0)

In [44]:
user_poi_if_buy

,user_id,wm_poi_id,if_buy
0,0,1806,0.0
1,0,970,0.0
2,0,1945,0.0
3,0,708,0.0
4,0,2366,0.0
...,...,...,...
2999998,999,812,0.0
2999999,999,2120,0.0
3000000,999,1781,0.0
3000001,999,2624,0.0


In [45]:
user_poi_if_buy['if_buy'].unique()

array([0., 1.])

In [ ]:
# 先安装推荐系统库 surprise，在anaconda中运行：
# conda install -c conda-forge scikit-surprise

In [46]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [47]:
reader = Reader()

In [48]:
data = Dataset.load_from_df(user_poi_if_buy, reader)
# data.split(n_folds=5)

In [50]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
ratings[ratings['userId'] == 1]

In [ ]:
svd.predict(1, 302, 3)

## Embedding

In [51]:
user_poi_if_buy

,user_id,wm_poi_id,if_buy
0,0,1806,0.0
1,0,970,0.0
2,0,1945,0.0
3,0,708,0.0
4,0,2366,0.0
...,...,...,...
2999998,999,812,0.0
2999999,999,2120,0.0
3000000,999,1781,0.0
3000001,999,2624,0.0
